In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)     # Linear = fully connected(fc) = dense
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)                   # 卷积层中不包含激活函数，需要手动添加
        t = F.max_pool2d(t, kernel_size=2, stride=2)     # 池化操作（没有权重的函数不能称为层）
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12*4*4)       # 必须手动flatten
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        # t = F.softmax(t, dim=1)           # 不直接在forward中用softmax，而是在训练过程中用cross-entropy损失函数计算loss，其中自带softmax
        
        return t

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [4]:
torch.set_grad_enabled(False)            # 关闭自动计算梯度

In [5]:
network = Network()

In [10]:
sample = next(iter(train_set))

In [11]:
image, label = sample
image.shape

torch.Size([1, 28, 28])

In [12]:
image.unsqueeze(0).shape           # 增加一个长度为1的轴(batch_size)

torch.Size([1, 1, 28, 28])

In [13]:
pred = network(image.unsqueeze(0))   # 将图像送入网络进行计算,返回值就是forward函数的返回值

In [14]:
pred.shape

torch.Size([1, 10])

In [15]:
pred

tensor([[-0.0860, -0.0837,  0.1099,  0.0447, -0.0141, -0.0006,  0.0194,  0.1543,  0.0808, -0.0027]])

In [16]:
label

9

In [17]:
pred.argmax(dim=1)   # 得到概率最高的下标

tensor([7])

In [18]:
F.softmax(pred, dim=1)

tensor([[0.0895, 0.0897, 0.1089, 0.1020, 0.0962, 0.0975, 0.0994, 0.1138, 0.1057, 0.0973]])

In [19]:
F.softmax(pred, dim=1).sum()

tensor(1.)

In [6]:
data_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=10
)

In [7]:
batch = next(iter(data_loader))

In [8]:
images, labels = batch

In [12]:
images.shape

torch.Size([10, 1, 28, 28])

In [13]:
labels.shape

torch.Size([10])

In [14]:
preds = network(images)
preds.shape

torch.Size([10, 10])

In [15]:
preds

tensor([[ 0.1183, -0.0298,  0.0083, -0.1290, -0.1002,  0.1151, -0.0808,  0.1337, -0.0231,  0.1287],
        [ 0.1093, -0.0375, -0.0038, -0.1295, -0.1037,  0.1090, -0.0887,  0.1391, -0.0249,  0.1310],
        [ 0.1270, -0.0301,  0.0022, -0.1260, -0.0985,  0.1066, -0.0920,  0.1343, -0.0215,  0.1273],
        [ 0.1228, -0.0339,  0.0022, -0.1259, -0.0993,  0.1070, -0.0876,  0.1331, -0.0210,  0.1286],
        [ 0.1188, -0.0399,  0.0014, -0.1235, -0.0943,  0.1038, -0.0800,  0.1332, -0.0251,  0.1239],
        [ 0.1172, -0.0325, -0.0038, -0.1281, -0.1088,  0.1150, -0.0889,  0.1373, -0.0246,  0.1347],
        [ 0.1253, -0.0301, -0.0082, -0.1352, -0.0990,  0.1145, -0.1098,  0.1515, -0.0299,  0.1392],
        [ 0.1190, -0.0277,  0.0025, -0.1309, -0.1088,  0.1205, -0.0845,  0.1327, -0.0209,  0.1325],
        [ 0.1270, -0.0276,  0.0015, -0.1222, -0.1035,  0.1126, -0.0979,  0.1401, -0.0207,  0.1341],
        [ 0.1288, -0.0256,  0.0010, -0.1188, -0.1019,  0.1137, -0.0947,  0.1361, -0.0185,  0.1335]])

In [16]:
preds.argmax(dim=1)

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [18]:
labels

tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

In [19]:
preds.argmax(dim=1).eq(labels)          # 与labels张量逐个比对，相同则为True

tensor([False, False, False, False, False, False,  True, False, False, False])

In [20]:
preds.argmax(dim=1).eq(labels).sum()    # 预测正确的个数

tensor(1)

In [21]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [22]:
get_num_correct(preds, labels)

1